In [1]:
import numpy as np

In [2]:
from sklearn.datasets import fetch_mldata

In [3]:
mnist = fetch_mldata('MNIST original')

/Users/davescott/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)
/Users/davescott/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)


In [42]:
X, y = mnist["data"[:70000]], mnist["target"[:70000]]

In [43]:
from sklearn.model_selection import train_test_split

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.142857)

In [45]:
shuffle_index=np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

# Random Forest Classifier and Training before applying PCA

In [46]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=50, max_leaf_nodes=32, n_jobs=-1, random_state=30)
rnd_clf2 = RandomForestClassifier(n_estimators=50, max_leaf_nodes=32, n_jobs=-1, random_state=30)

In [47]:
import time

In [48]:
start = time.time()
rnd_clf.fit(X_train, y_train)
end = time.time()
print(end - start)

2.4809019565582275


### our model takes 2.4 seconds to train without dimensionality reduction.

In [49]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(X_train)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [50]:
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1
d

154

In [51]:
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X_train)

In [52]:
start = time.time()
rnd_clf2.fit(X_reduced, y_train)
end = time.time()
print(end - start)

4.04552698135376


### We can see for our reduced dimension dataset the training is actually substantially longer. This usually is due to the dataset itself. Some datasets are better than others for dimensionality reduction.

In [54]:
from sklearn.metrics import accuracy_score
X_test_reduced = pca.transform(X_test)
y_pred = rnd_clf.predict(X_test)
y_pred2 = rnd_clf2.predict(X_test_reduced)

print(accuracy_score(y_pred, y_test))
print(accuracy_score(y_pred2, y_test))

0.8552
0.8265


## A slight decrease in performance, as well as slowing down training. This is a single case, but it's important to remember that in other settings we may be able to have much more time saved for training with a minimal decrease in model performance.